#### External vlaidation
#### This notebook checks node connectivity change by iterating random groups and calculating the mean median CVS of node-group connections (between the node and all nodes in the group, in the form of node pairs).
#### A histogram of mean CVS is plotted and the scores of reseection nodes are marked on the histogram.
##### A range of CVS scores - a standard deviation around the mean score - captures the resection in all subjects.
##### However, non-resected nodes are also included.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/gdrive')
main_folder = "/content/gdrive/My Drive/Ext-val/"

df = pd.read_csv(main_folder + f"cvs_pairs_ext.csv")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
connectivity_measures = list(df['CM'].unique())
print(connectivity_measures)

['PAC', 'SCR-(1,4)', 'SCI-(1,4)', 'PLV-(1,4)', 'PLI-(1,4)', 'CC-(1,4)', 'SCR-(4,8)', 'SCI-(4,8)', 'PLV-(4,8)', 'PLI-(4,8)', 'CC-(4,8)', 'SCR-(8,13)', 'SCI-(8,13)', 'PLV-(8,13)', 'PLI-(8,13)', 'CC-(8,13)', 'SCR-(13,30)', 'SCI-(13,30)', 'PLV-(13,30)', 'PLI-(13,30)', 'CC-(13,30)', 'SCR-(30,70)', 'SCI-(30,70)', 'PLV-(30,70)', 'PLI-(30,70)', 'CC-(30,70)', 'SCR-(70,150)', 'SCI-(70,150)', 'PLV-(70,150)', 'PLI-(70,150)', 'CC-(70,150)']


In [7]:
from inspect import ismethod
from datetime import timedelta as _time
from datetime import datetime
from collections.abc import Iterable as iterable

def some(field): return (field is not None and field != [] and field != {} and field != ()) or field == True
def no(field): return not some(field) or field==False or field==''

class class_of:
    _instance = None
    def __init__(_, object):
        _._is = type(object)
    def inherits(_, *types):
        return issubclass(_._is, types)
    def has(_, *types): return _.inherits(*types)
    def __enter__(self):
        self._instance = self
        return self
    def __exit__(self, type, value, traceback): self._instance = None
    @staticmethod
    def each_in(list):
        if isiterable(list):
            return [type(item) for item in list]

class struct:
    def __init__(table, **sets): table.__dict__.update(sets)
    @property
    def sets(this): return set(dir(this)) - set(dir(type(this)))
    def set(object, **fields):
        for field in fields: setattr(object, field, fields[field])
    def get(object, *fields): return [getattr(object, field) for field in fields if field in object.__dict__]
    def _clonable(set, mask=None):
        check = set.__dict__.copy()
        clonable = check.copy()
        if some(mask): pass
#            for field in check:
#                if sum([int(_(check[field])) for _ in mask])+sum([int(_(field)) for _ in mask]): clonable.pop(field)
        return clonable
    @staticmethod
    def _from(type):
        if hasattr(type, '__dict__'): return struct(**type.__dict__.copy())
        return struct()

def meta(data, *mask): return struct._from(data)._clonable(mask)
def get(data, *fields):
    if not issubclass(type(data), dict): data=struct._from(data)._clonable()
    return struct(**data).get(*fields)

class table(struct):
    def _default(field, name, value):
        try: return getattr(field, name)
        except: setattr(field, name, value)
        return value
    def clear(this, *fields):
        sets = this.sets
        if not fields: fields = sets
        if fields:
            set = [field for field in fields if hasattr(this,field) and not ismethod(getattr(this, field))]
            for field in set: delattr(this, field)
    def has(this, *fields):
        return all([hasattr(this, field) for field in fields])
    def has_not(this, *fields): return not this.has(*fields)
    def check(this, **KV):
        try: check = [KV[key]==this.__dict__[key] for key in KV]
        except: return False
        return all(check)
    def find(this, _type):
        return [value for value in this.sets if class_of(get(this,value)[0]).inherits(_type)]
    def clone(this):
        clone = type(this)()
        sets = this._clonable()
        clone.set(**sets)
        return clone

def isiterable(this): return isinstance(this, iterable) and type(this) is not str
def default(field, name, value): return table(**field)._default(name, value)

def ni(list):
    if isiterable(list):
        for n,i in enumerate(list): yield n,i
    else:
        for n,i in enumerate(list.__dict__.keys()): yield n,i

class at(table):
    DAY, HOUR, MIN = 86400, 3600, 60
    def __init__(_, dtime=None, **sets):
        _.set(**sets)
        if some(dtime) and issubclass(type(dtime), _time): _._time = dtime
        else:
            d,h,m,s,ms = _._default('d',0), _._default('h',0), _._default('m',0), _._default('s',0), _._default('ms',0)
            if not any([d,h,m,s,ms]): now=datetime.now(); _._time = now-datetime(now.year, now.month, now.day)
            else: _._time = _time(days=d, hours=h, minutes=m, seconds=s, milliseconds=ms)
        _.clear('d','h','m','s','ms')
    def __sub__(_, dtime):
        of=type(dtime); sets=_._clonable()
        if issubclass(of, _time): return at(_._time-dtime, **sets)
        elif issubclass(of, at): sets.update(dtime._clonable()); return at(_._time-dtime._time, **sets)
    def __add__(_, dtime):
        of=type(dtime); sets=_._clonable()
        if issubclass(of, _time): return at(_._time+dtime, **sets)
        elif issubclass(of, at): sets.update(dtime._clonable()); return at(_._time+dtime._time, **sets)
    def __str__(_): return str(_._time)
    @property
    def seconds(_): return _._time.seconds
    @property
    def S(_): return _.seconds
    @property
    def minutes(_): return _._time.seconds/60
    @property
    def M(_): return _.minutes
    @property
    def hours(_): return _.minutes/60
    @property
    def H(_): return _.hours
    @property
    def days(_): return _._time.days
    @property
    def D(_): return _.days
    @staticmethod
    def zero(): return at(_time())

from inspect import isfunction, ismethod, isgeneratorfunction, isgenerator, isroutine
from inspect import isabstract, isclass, ismodule, istraceback, isframe, iscode, isbuiltin
from inspect import ismethoddescriptor, isdatadescriptor, isgetsetdescriptor, ismemberdescriptor
from inspect import isawaitable, iscoroutinefunction, iscoroutine

from collections.abc import Iterable as iterable

import pickle

def isfx(field): return ismethod(field) or isfunction(field)

class GhostSet:
    """ enhanced interface (ghost) to retrieve class fields """
    def _meta(data): return {k:v for k,v in data.__dict__.items() if not isfx(v)}
    def _at_last(_, sets): pass
    def _set(object, **sets):
        ''' use to fast initialize fields | needed to avoid initialization problems at copy by value '''
        for field in sets: setattr(object, field, sets[field])
        object._at_last(sets)
GSet = GhostSet

def meta(object):
    ''' retrieves clonable object metadata (__dict__) as a copy '''
    if isinstance(object, GSet): return object._meta()
    return {}

class ClonableObjectGhost:
    """ enhanced interface (ghost) for clonable objects """
    def _by_val(_, depth=-1, _layer=0): pass
GCo = ClonableObjectGhost

class ClonableObject(GSet, GCo):
    """ base clonable object """
    def __init__(this, **data): this._set(**data)
    def __call__(_, **options): _._set(**options)
    def _by_val(_, depth=-1, _layer=0):
        copy = type(_)()
        copy._set(**_._meta())
        if depth<0 or depth>_layer:
            for field in copy.__dict__:
                if isinstance(copy.__dict__[field], ClonableObjectGhost):
                    copy.__dict__[field] = copy.__dict__[field]._by_val(depth,_layer+1)
        return copy
COb = ClonableObject

def copy_by_val(object, depth=-1, _layer=0):
    if isinstance(object, GCo): return object._by_val(depth,_layer)
    return object
copy = by_val = vof = copy_by_val

class ComparableGhost:
    """ enhanced interface (ghost) for comparing instances """
    def _compare(a, b):
        if type(a) != type(b): return False
        if a.__dict__ == b.__dict__: return True
        return False
    def __eq__(a, b): return a._compare(b)
GEq = ComparableGhost

class IterableObjectGhost(GSet):
    """ enhanced interface (ghost) for iterables: exposes __dict__,
        therefore Iterable Objects are like lua dictionaries """
    def __contains__(this, key): return key in this.__dict__
    def __iter__(this): return iter(this.__dict__)
    def items(my): return my.__dict__.items()
    def __getitem__(by, field): return by.__dict__[field]
    def __setitem__(by, field, value): by.__dict__[field] = value
    def pop(by, field): return by.__dict__.pop(field)
GIo = IterableObjectGhost

class ReprGhost:
    """ enhanced interface (ghost) for the skeleton method _repr,
        see implementation of Struct for a working example;
        Record __repr__ override uses _lines_ for max lines display """
    _lines_ = 31
    _chars_ = 13
    _msgsz_ = 62
    _ellipsis_ = ' ... '
    def _repr(my, value):
        _type = ''.join(''.join(str(type(value)).split('class ')).split("'"))
        _value = '{}'.format(value)
        if len(_value)>my._chars_:
            show = int(my._chars_/2)
            _value = _value[:show]+my._ellipsis_+_value[-show:]
        return '{} {}'.format(_type, _value)
    def _resize(this, message, at=.7):
        if len(message)>this._msgsz_:
            start = int(at*this._msgsz_)
            end = this._msgsz_-start
            return message[:start]+this._ellipsis_+message[-end:]
        return message
GRe = ReprGhost

def set_repr_to(lines): GRe._lines_ = lines

class Struct(COb, GEq, GIo, GRe):
    """ structured autoprintable object, behaves like a lua dictionary """
    def __repr__(_):
        return '\n'.join(['{}:\t{}'.format(k, _._repr(v)) for k,v in _.items()])
struct = Struct

class RecordableGhost:
    """ enhanced interface (ghost) for type recording,
        see Record for a working example """
    @staticmethod
    def load(filename):
        with open(filename, 'rb') as file: return pickle.load(file)
    def save(data, filename):
        with open(filename, 'wb') as file: pickle.dump(data, file)

GRec = RecordableGhost

class Record(GSet, GCo, GRec, GEq, GRe):
    """ wrapper for any object or value, auto-inspects and provides load/save type structure """
    data = None
    _check = dict(
            isfunction=isfunction, ismethod=ismethod, isgeneratorfunction=isgeneratorfunction, isgenerator=isgenerator, isroutine=isroutine,
            isabstract=isabstract, isclass=isclass, ismodule=ismodule, istraceback=istraceback, isframe=isframe, iscode=iscode, isbuiltin=isbuiltin,
            ismethoddescriptor=ismethoddescriptor, isdatadescriptor=isdatadescriptor, isgetsetdescriptor=isgetsetdescriptor, ismemberdescriptor=ismemberdescriptor,
            isawaitable=isawaitable, iscoroutinefunction=iscoroutinefunction, iscoroutine=iscoroutine
                   )
    def __init__(this, token, **meta):
        this.data = token
        this.__dict__.update({k:v(token) for k,v in this._check.items()})
        super()._set(**meta)
    @property
    def type(_): return type(_.data)
    def inherits(_, *types): return issubclass(_.type, types)
    @property
    def isbaseiterable(_): return _.inherits(tuple, list, dict, set) or _.isgenerator or _.isgeneratorfunction
    @property
    def isiterable(_): return isinstance(_.data, iterable) and _.type is not str
    def _clone_iterable(_):
        if _.inherits(dict): return _.data.copy()
        elif _.isgenerator or _.isgeneratorfunction: return (i for i in list(_.data))
        else: return type(_.data)(list(_.data)[:])
    def _meta(data): return {k:v for k,v in data.__dict__.items() if k != 'data' and not isfx(v)}
    def _by_val(_, depth=-1, layer=0):
        data = _.data
        if _.isiterable: data = _._clone_iterable()
        elif _.inherits(ClonableObjectGhost): data = by_val(data, depth, layer)
        return type(_)(data, **meta(_))
    def __enter__(self): self._instance = self; return self
    def __exit__(self, type, value, traceback): self._instance = None
    def __repr__(self):
        if not hasattr(self, '_preprint'): return Record(self.data, _preprint='', _lines=Record(Record._lines_)).__repr__()
        if self.isbaseiterable:
            pre, repr = self._preprint, ''
            for n,i in enumerate(self.data):
                if self._lines.data == 0: break
                else: self._lines.data -= 1
                index, item = str(n), i
                if self.inherits(dict): index += ' ({})'.format(str(i)); item = self.data[i]
                repr += pre+'{}: '.format(index)
                next = Record(item, _preprint=pre+'\t', _lines=self._lines)
                if next.isiterable: repr += '\n'
                repr += next.__repr__()
                repr += '\n'
            return repr
        elif self.inherits(GCo): return Record(self.data._meta(), _preprint=self._preprint, _lines=self._lines).__repr__()
        else: return self._repr(self.data)
REc = Record

class Bisect(list, COb):
    """ bisect implementation using clonable objects """
    def __init__(set, *items, key=None, reverse=False):
        if not key: key = lambda  x:x
        super().__init__(sorted(items, reverse=reverse, key=key))
    def _bisect(set, item, key, reverse, bottom, top):
        def _(check):
            if key: return key(check)
            return check
        at = int((top-bottom)/2)+bottom
        if len(set)==0: return (0,-1)
        if item==_(set[at]): return (at,0)
        bigger = item<_(set[at])
        if bigger != reverse:
            if at-bottom>0: return set._bisect(item, key, reverse, bottom, at)
            return (at,-1)
        elif top-at>1: return set._bisect(item, key, reverse, at, top)
        return (at,1)
    def search(_, item, key=None, reverse=False):
        if not key: key = lambda x:x
        return _._bisect(item, key, reverse, 0, len(_))
    def _by_val(_, depth=-1, _layer=0):
        copy = super()._by_val(depth, _layer)
        copy += _[:]
        return copy
BSx = Bisect

In [8]:
def get_non_operated(main_folder):
    try:
        # Read the Excel file
        df = pd.read_excel(main_folder + 'metadata.xlsx')
        # Filter rows where OUTCOME is -1
        outcome_minus_one = df[df['OUTCOME'] == -1]
        # Get the SCC_ID values from the filtered rows
        nonop = outcome_minus_one['SCC_ID'].tolist()
        print(nonop)
        return nonop

    except Exception as e:
        print(f"Error: {e}")
        return None


def get_outcome_for_subject(sub, main_folder):
    try:
        df = pd.read_excel(main_folder + 'metadata.xlsx')
        # Find the row corresponding to the subject ID
        row = df[df['SCC_ID'] == sub]
        # Get the outcome from the row
        outcome = row['OUTCOME'].values[0] if not row.empty else None
        return outcome

    except Exception as e:
        print(f"Error: {e}")
        return None

def get_soz_labels(sub, main_folder):
    try:
        df = pd.read_excel(main_folder + "metadata.xlsx")

        # Find the row corresponding to the given sub
        row = df[df['SCC_ID'] == sub]
        # Extract the SOZ labels from the row
        soz_labels = row['SOZ'].values[0]
        # Split the string by comma and remove spaces
        soz_labels = soz_labels.replace(' ', '').split(',')
        # Convert all letters to uppercase
        soz_labels = [label.upper() for label in soz_labels]
        return soz_labels

    except Exception as e:
        print(f"Error: {e}")
        return None



Helper functions:

In [9]:
def MM(x):
    return (np.min(x)+np.max(x))/np.mean(x)


def plot_scores_hist(data):
    # Plot a histogram of the 'Score' column
    plt.hist(data['Score'], bins=20, edgecolor='grey', color='lightgrey')
    plt.xlabel('Score')
    plt.ylabel('Frequency')
    plt.title('Histogram of Mean CVS for All Groups')

    # Calculate mean, median, and standard deviation
    mean_score = data['Score'].mean()
    std_score = data['Score'].std()

    # Plot vertical lines for mean, median, 1 std, 2 std, and 3 std
    plt.axvline(mean_score, color='r', linestyle='dashed', linewidth=2, label='Mean')
    plt.axvline(mean_score + std_score, color='b', linestyle='dashed', linewidth=2, label='1 Std Dev')
    plt.axvline(mean_score + 2*std_score, color='y', linestyle='dashed', linewidth=2, label='2 Std Dev')
    plt.axvline(mean_score + 3*std_score, color='c', linestyle='dashed', linewidth=2, label='3 Std Dev')
    plt.axvline(mean_score + 4*std_score, color='m', linestyle='dashed', linewidth=2, label='4 Std Dev')

    plt.legend()
    plt.show()


def plot_hist_en(node_mean_scores, resection_scores):
    # Create a histogram of all scores
    plt.hist(node_mean_scores['Score'], bins=10, edgecolor='black', alpha=0.7, label='All Scores')

    # Add vertical lines to mark the scores of resection nodes
    for score in resection_scores:
        plt.axvline(score, color='red', linestyle='dashed', linewidth=2)

    plt.xlabel('Scores')
    plt.ylabel('Frequency')
    plt.title('Histogram of Scores with Resection Scores Marked')
    plt.show()


# Function to check for overlap
def compute_overlap(group1, group2):
    return list(set(group1).intersection(group2))

# Create a function to compute the percentage of overlap
def compute_overlap_percentage(group, resection_nodes):
    overlap_count = sum(1 for node in group if node in resection_nodes)
    overlap_percentage = (overlap_count / len(group))
    return overlap_percentage

def jaccard_index(group, resection):
    """
    Calculate the Jaccard index for the overlap between a group and a resection.

    Args:
    group (set or list): A set or list of nodes in the group.
    resection (set or list): A set or list of nodes in the resection.

    Returns:
    float: The Jaccard index, a value between 0 and 1.
    """
    # Convert input to sets for efficient set operations
    group_set = set(group)
    resection_set = set(resection)

    # Calculate the size of the intersection and union of the sets
    intersection_size = len(group_set.intersection(resection_set))
    union_size = len(group_set.union(resection_set))

    # Calculate the Jaccard index
    if union_size == 0:
        return 0.0  # Handle the case when the union is empty
    else:
        return intersection_size / union_size


In [10]:
import re

# Dictionary for band symbol substitutions
band_subs = {
    "0,4": "δ",
    "4,8": "ϴ",
    "8,13": "α",
    "13,30": "β",
    "30,70": "low γ",
    "70,150": "high γ"
}

# Dictionary for connectivity method substitutions
cm_subs = {
    "SCR-": "Real spectral coherence",
    "SCI-": "Imaginary spectral coherence",
    "PLV-": "Phase-locking value",
    "PLI-": "Phase lag index",
    "CC-": "Cross-correlation",
    "PAC-": "PAC",
    "PEC-": "PEC"
}

# Function to replace bands with their corresponding symbols
def substitute_bands(s):
    # Find all occurrences of band ranges in the format "(a,b)"
    bands = re.findall(r'\(\d+,\d+\)',s)

    # Substitute each band range with its corresponding symbol
    for band in bands:
        band_range = band[1:-1]  # Remove parentheses
        if band_range in band_subs:
            s = s.replace('-','').replace(band, f" ({band_subs[band_range]})")

    return s

# Function to replace connectivity methods with their corresponding names
def substitute_cm(s):
    # Find all occurrences of connectivity method prefixes
    for cm in cm_subs:
        s = re.sub(r'\b' + cm + r'\b', cm_subs[cm], s)

    return s

# Test
input_str = "CC-(30,70), SCR-(70,150)"
output_str = substitute_cm(input_str)
output_str = substitute_bands(output_str)

print(output_str)


CC (low γ), SCR (high γ)


44-EpiGAME

In [12]:
connectivity_measures = ['PLI-(4,8)', 'PLI-(70,150)']

In [ ]:
import math
from scipy import stats

# Set a specific random seed (e.g., 42 for reproducibility)
random_seed = 42
np.random.seed(random_seed)

subs = list(range(1,28))

for sub in subs:

  if sub not in [str(id) for id in get_non_operated(main_folder)]:

    surgical_outcome = get_outcome_for_subject(int(sub), main_folder)
    print(surgical_outcome)

    # we need resection_nodes and group_size to check the winners
    all_nodes = REc.load(main_folder + f"result/{sub}-PAC.res").data.data.nodes

    resection_nodes = get_soz_labels(int(sub), main_folder)

    ratio_resected = len(resection_nodes) / len(all_nodes) # for later normalization
    print(f"Ratio of resected nodes = {ratio_resected}")

    df_sub = df[df['Subject'] == sub]

    for measure in connectivity_measures:
      df_sub_cm = df_sub[df_sub['CM'] == measure]

      r = 0.1
      group_size = int(len(all_nodes)*r)
      print("Random group size =", group_size)

      groups, mm_cvs = [],[]

      for i in range(len(all_nodes)*5):

          # random groups are created in each iteration
          group_labels = sorted(list(np.random.choice(all_nodes, size=group_size, replace=False)))

          if group_labels not in groups:

            # Split the Labels column into two nodes using '<->'
            split_labels = df_sub_cm['Labels'].str.split('<->', expand=True)

            # Check if both nodes are in group_labels
            relevant_rows = df_sub_cm[((split_labels[0].isin(group_labels)) & split_labels[1].isin(group_labels))]
            # Calculate (max(CVS)+min(CVS)) / mean(CVS) scores (MM scores)
            mm_cvs.append(list(relevant_rows['CVS'].apply(lambda x: MM(list(map(float, x.strip('[]').split()))))))
            groups.append(group_labels)

      data = {
          'Group': groups,
          'Score': [np.median(x) for x in mm_cvs]} # median MM score for a random group

      data = pd.DataFrame(data)

      # Count rows with NaN values
      nan_rows_count = data.isna().sum().sum()
      print("Number of Rows with NaN Values:", nan_rows_count)
      # Drop NaN rows
      data = data.dropna()

      plot_scores_hist(data)

      # # Find the common nodes (overlap)
      # data['Overlap_with_Resection'] = data['Group'].apply(lambda row: compute_overlap(row, resection_nodes))

      # # Apply the function to each row
      # data['Overlap_Percentage'] = data['Group'].apply(lambda row: compute_overlap_percentage(row, resection_nodes))

      # Define score ranges relative to standard deviations of MM score distribution
      # Calculate the median and standard deviation of MM score
      median_score = data['Score'].median()
      std_dev_score = data['Score'].std()

      # Define the number of standard deviations to consider
      num_std_devs = 7
      # Create score ranges relative to median and standard deviation
      score_ranges = []
      min_std, max_std = num_std_devs, -num_std_devs
      # Create an array of x-values with an offset
      x_values = []

      for i in range(-num_std_devs*2, num_std_devs*2, 1):
          range_start = median_score + i * std_dev_score /2
          range_end = median_score + (i + 1) * std_dev_score /2
          # Check if there are score values in the range, and only add the range if there are values
          if any((range_start <= data['Score']) & (data['Score'] < range_end)):
              score_ranges.append((range_start, range_end))
              min_std = min(min_std, i)
              max_std = max(max_std, i)
              x_values.append(i+0.5)

      # Initialize lists
      overlap_percentages = []
      best_score_range = None
      best_overlap_percentage = 0.0
      all_group_overlaps = []

      for score_range in score_ranges:

          group_overlaps = []
          lower_bound = score_range[0]
          upper_bound = score_range[1]

          # Filter groups within the current score range
          filtered_groups = data[(data['Score'] >= lower_bound) & (data['Score'] < upper_bound)]

          # Calculate overlap scores and create a new "Group_Overlap" column
          data['Group_Overlap'] = 0  # Initialize the column with zeros

          # Compute the overlap between the filtered groups
          group_overlap_scores = []

          if len(filtered_groups) == 1:

            resection_overlap = compute_overlap_percentage(filtered_groups.iloc[0]['Group'], resection_nodes)
            group_overlaps.append(resection_overlap)

          else:

            for i, group1 in filtered_groups.iterrows():
                for j, group2 in filtered_groups.iterrows():

                    if group1['Group'] != group2['Group']:
                        # # 1) Overlap between groups' intersection and the resection or Jaccard Index
                        group_intersection = compute_overlap(group1['Group'], group2['Group'])
                        resection_overlap = compute_overlap_percentage(group_intersection, resection_nodes) if group_intersection else 0
                        group_overlap_scores.append(resection_overlap)

                group_overlaps += [np.mean(group_overlap_scores)] # save the mean overlap of group1

          all_group_overlaps.append(group_overlaps)

      # Check if all lists contain only zeros
      all_zeros = [all(x == 0 for x in sublist) for sublist in all_group_overlaps]
      # Check if all elements in 'all_zeros' list are True
      if all(all_zeros): print("All lists contain only zeros")

      else:
        # Plot overlap percentages across score ranges
        plt.figure(figsize=(10, 6))
        plt.boxplot(all_group_overlaps, labels=x_values, positions=x_values, showfliers=True)

        # Add the scatter plot
        for i, x in enumerate(x_values):
            y = all_group_overlaps[i]
            x_coords = np.random.normal(x, 0.04, size=len(y))  # Set x-coordinates to position on the x-axis
            plt.scatter(x_coords, y, marker='o', alpha=0.5, label=x)
            # Annotate the number of samples next to each boxplot
            plt.text(x, plt.ylim()[1], f'{len(y)}', ha='center', va='bottom', size='large')

        plt.ylabel('Resection Overlap', size=20)
        plt.xlabel(f'ΔC Score Distribution Ranges: {substitute_cm(substitute_bands(measure))}', size=20)

        # Set custom x-axis ticks at points between the ranges
        plt.xticks(x_values)
        # Add vertical lines for median and each standard deviation
        borders = x_values+[x_values[-1]+1]
        for k in [int(x-0.5) for x in borders]:
            label = 'M' if k == 0 else f'{k/2}σ'
            plt.axvline(x=k, color='gray', linestyle='--', linewidth=0.5)
            plt.text(k, plt.ylim()[0], label, horizontalalignment='left', size='large')
        plt.yticks(fontsize=14)
        plt.xticks([])  # Remove x-axis ticks
        plt.show()